In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import pprint
import json
import re
from api_key import openweather_api
from datetime import date, datetime

In [2]:
airport_name_url = 'https://gist.githubusercontent.com/tdreyno/4278655/raw/7b0762c09b519f40397e4c3e100b097d861f5588/airports.json'
airport_name_source = requests.get(airport_name_url).json()
airport_name_source

[{'code': 'AAA',
  'lat': '-17.3595',
  'lon': '-145.494',
  'name': 'Anaa Airport',
  'city': 'Anaa',
  'state': 'Tuamotu-Gambier',
  'country': 'French Polynesia',
  'woeid': '12512819',
  'tz': 'Pacific/Midway',
  'phone': '',
  'type': 'Airports',
  'email': '',
  'url': '',
  'runway_length': '4921',
  'elev': '7',
  'icao': 'NTGA',
  'direct_flights': '2',
  'carriers': '1'},
 {'code': 'AAE',
  'lat': '36.8236',
  'lon': '7.8103',
  'name': 'El Mellah Airport',
  'city': 'El Tarf',
  'state': 'Annaba',
  'country': 'Algeria',
  'woeid': '12510325',
  'tz': 'Africa/Algiers',
  'phone': '',
  'type': 'Airports',
  'email': '',
  'url': '',
  'runway_length': '9843',
  'elev': '16',
  'icao': 'DABB',
  'direct_flights': '6',
  'carriers': '2'},
 {'code': 'AAL',
  'lat': '57.0952',
  'lon': '9.85606',
  'name': 'Aalborg Airport',
  'city': 'Norresundby',
  'state': 'Nordjylland',
  'country': 'Denmark',
  'woeid': '12512587',
  'tz': 'Europe/Copenhagen',
  'phone': '',
  'type': 'Air

In [3]:
#example origin airport name [2024-07-21 for 'Miami International'] has rain
origin_airport = 'Miami International Airport'

In [4]:
for i in range(len(airport_name_source)):
    name_of_airport = airport_name_source[i]['name']
    lat = airport_name_source[i]['lat']
    long = airport_name_source[i]['lon']
    if name_of_airport == origin_airport:
            o_lat = lat
            o_long = long

In [9]:
#insert example flight_date [2024-07-21 for 'Miami International'] has rain
flight_date = input('Please enter date of flight in YYYY-MM-DD format')

Please enter date of flight in YYYY-MM-DD format 2024-07-21


In [10]:
flight_date

'2024-07-21'

In [11]:
#get weather data on openweather api
url = f'https://api.openweathermap.org/data/2.5/forecast?lat={o_lat}&lon={o_long}&appid={openweather_api}'

In [12]:
url

'https://api.openweathermap.org/data/2.5/forecast?lat=25.7953&lon=-80.2727&appid=8e6f6a4e6412e1d2c4ee8b4447c90d96'

In [14]:
#get 7-day forecast for origin
origin_owforecast = requests.get(url).json()
pprint.pp(origin_owforecast)

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1721509200,
           'main': {'temp': 306,
                    'feels_like': 313,
                    'temp_min': 304.81,
                    'temp_max': 306,
                    'pressure': 1019,
                    'sea_level': 1019,
                    'grnd_level': 1017,
                    'humidity': 66,
                    'temp_kf': 1.19},
           'weather': [{'id': 802,
                        'main': 'Clouds',
                        'description': 'scattered clouds',
                        'icon': '03d'}],
           'clouds': {'all': 38},
           'wind': {'speed': 5.93, 'deg': 119, 'gust': 6.39},
           'visibility': 10000,
           'pop': 0,
           'sys': {'pod': 'd'},
           'dt_txt': '2024-07-20 21:00:00'},
          {'dt': 1721520000,
           'main': {'temp': 304.5,
                    'feels_like': 311.07,
                    'temp_min': 303.45,
                    'temp_max': 304.5,
 

In [30]:
origin_owforecast['list'][5]['rain']['3h']

0.14

In [34]:
#find precip
precip_in = 0
for i in range(len(origin_owforecast['list'])):
    weather_date = origin_owforecast['list'][i]['dt_txt']
    precip_prob = origin_owforecast['list'][i]['pop']
    match = re.search(f'^{flight_date}', weather_date)
    if match and precip_prob > 0:
        if 'rain' in origin_owforecast['list'][i]:
            precip_in = precip_in + origin_owforecast['list'][i]['rain']['3h']
    else:
        precip_in = precip_in + 0
precip_in

1.07

# Precipitation Function

In [5]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import pprint
import json
import re
from api_key import openweather_api
from datetime import date, datetime

def precip_fn(flight_date, origin_airport):
    #read in list of available airport names
    airport_name_url = 'https://gist.githubusercontent.com/tdreyno/4278655/raw/7b0762c09b519f40397e4c3e100b097d861f5588/airports.json'
    airport_name_source = requests.get(airport_name_url).json()

    #get lat long of airport from airport name input
    for i in range(len(airport_name_source)):
        name_of_airport = airport_name_source[i]['name']
        lat = airport_name_source[i]['lat']
        long = airport_name_source[i]['lon']
        if name_of_airport == origin_airport:
            o_lat = lat
            o_long = long
    
    #get weather data on openweather api
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={o_lat}&lon={o_long}&appid={openweather_api}'

    #get 7-day forecast for origin
    origin_owforecast = requests.get(url).json()

    #find precip
    precip_in = 0
    for i in range(len(origin_owforecast['list'])):
        weather_date = origin_owforecast['list'][i]['dt_txt']
        precip_prob = origin_owforecast['list'][i]['pop']
        match = re.search(f'^{flight_date}', weather_date)
        if match and precip_prob > 0:
            if 'rain' in origin_owforecast['list'][i]:
                precip_in = precip_in + origin_owforecast['list'][i]['rain']['3h']
        else:
            precip_in = precip_in + 0
    return {
        'precipitation': f'{precip_in} in.'
    }

In [7]:
precip_fn('2024-07-21', 'Miami International Airport')

KeyError: 'rain'

In [8]:
url

NameError: name 'url' is not defined